### 数据处理

In [2]:
import os
import pandas as pd

In [3]:
PATH = "F:\\badou\\tmp\\lr\\raw_data\\"

In [4]:
# data file
music_data = os.path.join(PATH, 'music_meta')
user_profile = os.path.join(PATH, 'user_profile.data')
user_watch_pref = os.path.join(PATH, 'user_watch_pref.sml')

In [5]:
# item description data
def gen_music_meta(nrows=None):
    df_music_meta = pd.read_csv(music_data,
                                sep='\001',
                                nrows=nrows,
                                names=['item_id', 'item_name', 'desc', 'total_timelen', 'location', 'tags'])
    del df_music_meta['desc']
    # 空值替换为 -
    return df_music_meta.fillna('-')

In [6]:
# user profile data
def gen_user_profile(nrows=None):
    return pd.read_csv(user_profile,
                       sep=',',
                       nrows=nrows,
                       names=['user_id', 'gender', 'age', 'salary', 'province'])

In [7]:
# user action data
def gen_user_watch(nrows = None):
    return pd.read_csv(user_watch_pref,
                       sep = '\001',
                       nrows = nrows,
                       names = ['user_id', 'item_id', 'stay_seconds', 'hour'])

In [8]:
gen_user_watch(1)

,user_id,item_id,stay_seconds,hour
0,01e069ed67600f1914e64c0fe7730944,4090309101,15,19


In [10]:
gen_music_meta(1)

,item_id,item_name,total_timelen,location,tags
0,93709100,韩国少女时代最新回归新专主打《I GOT A BOY》,304,-,"少女时代,鹿晗,韩国,exo综艺,最新回归,exo快乐大本营,吴世勋,fx组合,音悦台"


In [14]:
def user_item_score():
    """读取数据集 40000 条"""
    # 用户行为数据
    user_watch = gen_user_watch(40000)
    
    # 商品数据
    music_meta = gen_music_meta()
    
    # 将用户行为和商品数据相关联
    data = user_watch.merge(music_meta, how="inner", on="item_id")
    
    del user_watch
    del music_meta
    
    # 添加score = 用户收听的时间 / 商品的总时长
    data["score"] = data.apply(lambda x: float(x["stay_seconds"]) / float(x["total_timelen"]), axis=1)
    
    # 过滤数据只取相关的
    data = data[["user_id", "item_id", "score"]]
    
    # 对相同的user_id和item_id的打分求和(用户在不同时段多次收听)[求平均值也行]
    # groupby返回字典的形式，做 reset_index 变回 dataFrame 形式
    data = data.groupby(["user_id", "item_id"]).score.sum().reset_index()
    
    return data

In [15]:
data = user_item_score()
data.head()

c:\users\hedy\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2910: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,user_id,item_id,score
0,0000066b1be6f28ad5e40d47b8d3e51c,426100349,1.280000
1,000007042a11f469173bd85ff121eee9,9087609344,0.919811
2,00000cce929ed4de96d2cb6bbf956c53,251400320,1.025785
3,00000f7383b4ddb4885c7ef81b483e27,711800352,0.517241
4,00001fc17f67602470571d231a76176e,780800217,0.503356


In [16]:
def train_from_df(df, col_name = ['user_id', 'item_id', 'score']):
    """将数据集处理为 {user_id: {item_id: rating}} 格式"""
    train = dict()
    for _, row in data.iterrows():
        user_id = str(row[col_name[0]])
        item_id = str(row[col_name[1]])
        rating = row[col_name[2]]
        
        if train.get(user_id, -1) == -1:
            train[user_id] = dict()
        
        train[user_id][item_id] = rating
    
    return train

In [17]:
train = train_from_df(data, col_name = ['user_id', 'item_id', 'score'])

In [22]:
train["00016d63b5034c9565b3248f57dd3ba7"]

{'348800325': 0.8888888888888888, '350100326': 0.06302521008403361}

In [ ]:
# 写出
with open(train_file, 'w', encoding = 'utf-8') as f:
    f.write(str(train))